In [6]:
import pandas as pd
import numpy as np
import rasterio
import os

raster_path = "sugarcane_preprocessed.tif"

# Add robust error checking
if not os.path.exists(raster_path):
    raise FileNotFoundError(f"Raster file not found at: {raster_path}")

try:
    with rasterio.open(raster_path) as src:
        sugar_data = src.read(1)
        print(f"Raster loaded with shape: {sugar_data.shape}")
except Exception as e:
    raise RuntimeError(f"Failed to load raster: {e}")


FileNotFoundError: Raster file not found at: sugarcane_preprocessed.tif

In [ ]:
import rasterio
import numpy as np
import folium
from folium.raster_layers import ImageOverlay
from ipywidgets import interact, FloatSlider
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from scipy.ndimage import gaussian_filter

# Load sugarcane raster
sugar_path = raster_path

with rasterio.open(sugar_path) as src:
    sugar_raster = src.read(1)
    bounds = src.bounds

# Normalise to [0, 1]
sugar_norm = (sugar_raster - np.nanmin(sugar_raster)) / (np.nanmax(sugar_raster) - np.nanmin(sugar_raster))

# Set the centre point (site under question)
centre_lat = (bounds.top + bounds.bottom) / 2
centre_lon = (bounds.left + bounds.right) / 2

# Heatmap overlay function
def create_heatmap(threshold_percentile=5, blur_sigma=2):
    threshold_val = np.nanpercentile(sugar_norm, 100 - threshold_percentile)
    print(f"Showing top {threshold_percentile:.0f}% (threshold value: {threshold_val:.3f}) | Blur σ = {blur_sigma}")

    heat = np.where(sugar_norm >= threshold_val, sugar_norm, 0)
    heat_blurred = gaussian_filter(heat, sigma=blur_sigma)

    cmap = cm.plasma
    norm = mcolors.Normalize(vmin=threshold_val, vmax=np.nanmax(sugar_norm))
    rgba_img = cmap(norm(heat_blurred))

    alpha_mask = heat_blurred > threshold_val
    rgba_img[..., 3] = np.where(alpha_mask, 0.7, 0.0)

    img = Image.fromarray((rgba_img * 255).astype(np.uint8), mode="RGBA")
    temp_path = "/tmp/sugarcane_heatmap.png"
    img.save(temp_path)

    m = folium.Map(
        location=[centre_lat, centre_lon],
        zoom_start=10,
        tiles="CartoDB positron"
    )

    # Add heatmap overlay
    ImageOverlay(
        image=temp_path,
        bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
        opacity=1,
        name="Sugarcane Heatmap"
    ).add_to(m)

    # ✅ Add centre point marker
    folium.CircleMarker(
        location=[centre_lat, centre_lon],
        radius=6,
        color="#111111",
        fill=True,
        fill_color="#FFFFFF",
        fill_opacity=0.9,
        popup="Centre of Study Area"
    ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Two sliders: Top % and blur amount
interact(
    create_heatmap,
    threshold_percentile=FloatSlider(value=5, min=1, max=100, step=1, description='Top %:'),
    blur_sigma=FloatSlider(value=2, min=0, max=10, step=0.5, description='Blur σ:')
);


interactive(children=(FloatSlider(value=5.0, description='Top %:', min=1.0, step=1.0), FloatSlider(value=2.0, …